In [2]:
import sys
from rplidar import RPLidar
#from robocup import 
import numpy as np
import time
import math 
import cv2

robot_angle = -105



In [13]:
w = 500
lidar = RPLidar('/dev/ttyUSB0')
f = np.zeros((w, w), np.uint8)
cv2.imshow('img', f)
cv2.namedWindow('scale')
cv2.createTrackbar('scale_trac', 'scale', 1, 1000, no)
cv2.createTrackbar('line_sens', 'scale', 1, 150, no)
s = np.zeros((100, 300), np.uint8)
t1 = time.time()
accum = 0
while True:
    for scan in lidar.iter_scans():
        #lidar.stop()
        
        cv2.imshow('scale', s)
        scale = cv2.getTrackbarPos('scale_trac', 'scale')
        line_sens = cv2.getTrackbarPos('line_sens', 'scale')
        
        #print(len(scan))
        if time.time()-t1 > 1:
            print(accum, accum/time.time()-t1, "evo")
            t1 = time.time()
            accum = 0
        accum += len(scan)
        f, c = make_img_from_scan(scan)
                
        '''
        lines = cv2.HoughLine  s(f,1,np.pi/180,line_sens)
        #print(lines)
        if counter > 2 and lines is not None:
            delta = math.pi / 180 * 10
            #time.sleep(0.5)
            for rho,theta in lines[0]:
                #print(theta)
                #rho = abs(rho)
                if (theta < math.pi+delta and theta > math.pi-delta) or (theta > -delta and theta < delta):
                    a = np.cos(theta)
                    b = np.sin(theta)
                    x0 = a*rho
                    y0 = b*rho
                    x1 = int(x0 + 1000*(-b))
                    y1 = int(y0 + 1000*(a))
                    x2 = int(x0 - 1000*(-b))
                    y2 = int(y0 - 1000*(a))
                    cv2.line(f,(x1,y1),(x2,y2),255,2)
                    dist = pl_dist(w/2, w/2, rho, theta, scale)
                    dist2 = dist * (scale+0.01) / 50
                    print(dist, dist2)
        '''       
        '''
        cv2.imwrite('8.jpg', f)
        #f = cv2.dilate(f,None)
        gray = f #cv2.cvtColor(f,cv2.COLOR_BGR2GRAY)

        gray = np.float32(gray)
        dst = cv2.cornerHarris(gray,75,3,0.04)

        #result is dilated for marking the corners, not important
        dst = cv2.dilate(dst,None)

        # Threshold for an optimal value, it may vary depending on the image.
        f[dst>= dst.max()]=150
        #print(dst.max())'''

  
            
        cv2.imshow('img', f)
        if cv2.waitKey(5) != -1:
            cv2.destroyAllWindows()
            break
        
    
      
cv2.destroyAllWindows()
lidar.stop()

1084 -1574280033.600401 evo
2005 -1574280034.6499825 evo
2031 -1574280035.7316923 evo
2033 -1574280036.807159 evo
2037 -1574280037.886617 evo
2029 -1574280038.9653635 evo
2022 -1574280040.055905 evo
1850 -1574280041.1227121 evo
2027 -1574280042.1238885 evo
2016 -1574280043.190287 evo
1856 -1574280044.2670774 evo
2063 -1574280045.27493 evo
2047 -1574280046.3531663 evo
2051 -1574280047.4272623 evo
2057 -1574280048.5037158 evo
2049 -1574280049.599562 evo
1889 -1574280050.6709828 evo
2070 -1574280051.674901 evo
2071 -1574280052.7508492 evo
2047 -1574280053.8438642 evo
2048 -1574280054.924529 evo
1742 -1574280056.0032272 evo
1978 -1574280057.0836003 evo
2015 -1574280058.14871 evo
2032 -1574280059.2465599 evo
1875 -1574280060.3148599 evo


RPLidarException: Incorrect descriptor starting bytes

In [3]:
cv2.destroyAllWindows()
lidar.stop()


NameError: name 'lidar' is not defined

In [3]:
def no(x):
    a = 0

In [4]:
def make_img_from_scan(scan):
    w = 500;
    counter = 0
    f = np.zeros((w, w), np.uint8)
    n = len(scan)
    for i in range (n):
        m = scan[i]
        #print(m[0], m[1], m[2])
        x = w/2 + math.cos(math.radians(m[1]+robot_angle))*m[2]/(scale+0.01)*50
        y = w/2 + math.sin(math.radians(m[1]+robot_angle))*m[2]/(scale+0.01)*50
        if x < w-1 and y < w-1 and x>1 and y >1:
            f[int(y)][int(x)] = 255
            f[int(y)+1][int(x)] = 255
            f[int(y)-1][int(x)] = 255
            f[int(y)][int(x)+1] = 255
            f[int(y)][int(x)-1] = 255
    return f, counter

In [5]:
def pp_dist(x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

In [6]:
def pl_dist_polar(x, y, rho, theta):
    b = 1/math.cos(theta)*rho
    a = math.tan(theta)
    a = inf_t(a)
    absi = abs(a*x-y+b)
    root = math.sqrt(a**2+1)
    #print(theta, rho)
    #print(a, b, absi, root)
    return absi/root#*(scale+0.01)/50

In [7]:
def ps_dist(x, y, x1, y1, x2, y2):
    k0, l0 = line_from_points(x1, y1, x2, y2)
    k = -1/k0
    l1 = y1 - k*x1
    l2 = y2 - k*x2
    
    cp1 = (y > k*x + l1)
    cp2 = (y > k*x + l2)
    mid = (not cp1 and cp2) or (not cp2 and cp1)
    
    if mid:
        return pl_dist_cart(x, y, k0, l0)
    else:
        return min(pp_dist(x, y, x1, y1), pp_dist(x, y, x2, y2))
    
    
    

In [8]:
def inf_t(a):
    if a == float('inf'):
        a = 10000
    if a == float('-inf'):
        a = -10000
    return a

In [9]:
def pl_dist_cart(x, y, k, l):
    b = l
    a = k
    a = inf_t(a)
    absi = abs(a*x-y+b)
    root = math.sqrt(a**2+1)
    #print(theta, rho)
    #print(a, b, absi, root)
    return absi/root#*(scale+0.01)/50

In [10]:
def line_from_points(x1, y1, x2, y2):
    if x2 - x1 != 0:
        k = (y2-y1)/(x2-x1)
    else: 
        k = 10000*(y2-y1)
    l = y1 - k*x1
    return k, l

In [12]:


field = [
    [0, 0, 0, 1000],
    [0, 0, 1000, 0],
    [0, 1000, 1000, 1000],
    [1000, 0, 1000, 1000],
    [700, 300, 700, 700],
    [300, 300, 500, 700],
    [100, 100, 500, 100],
    [50, 900, 500, 800]
]

In [13]:
def get_inter(x, y, angles):
    for i in range (len(angles)):
        for j in range (len(field)):
            g =5
        

In [14]:
dist, hp = ps_inter(-10, 10, 15, [0, 0, 0, 500])
print(dist, hp)

NameError: name 'ps_inter' is not defined

In [15]:
def ps_inter(x, y, angle, seg):
    x1 = seg[0]
    y1 = seg[1]
    x2 = seg[2]
    y2 = seg[3]
    if angle == 0 or angle == 180:
        if x1 == x2:
            yp = y
            y11 = y1
            y22 = y2
            if y2 < y1:
                t = y11
                y11 = y2
                y22 = t
            if yp <= y11 or yp >= y22:
                return -1, [0, 0]
            if is_in_direction(x, y, x1, y, angle)==-1:
                return -1, [0, 0]
            return abs(x-x1), [x1, y]
        if y1 == y2:
            return -1, [0, 0]
        yp = y
        y11 = y1
        y22 = y2
        if y2 < y1:
            t = y11
            y11 = y2
            y22 = t
        if yp <= y11 or yp >= y22:
            return -1, [0, 0]
        k2, l2 = line_from_points(x1, y1, x2, y2)
        xp = (yp-l2)/k2
        if is_in_direction(x, y, xp, yp, angle)==-1:
            return -1, [0, 0]        
        return pp_dist(x, y, xp, yp), [xp, yp]
        
    if angle == 90 or angle == 270:
        if x1 == x2:
            return -1, [0, 0]
        if y1 == y2:
            xp = x
            x11 = x1
            x22 = x2
            if x2 < x1:
                t = x11
                x11 = x22
                x22 = t
            if xp <= x11 or xp >= x22:
                return -1, [0, 0]
            if is_in_direction(x, y, x, y1, angle)==-1:
                return -1, [0, 0]    
            return abs(y1-y), [x, y1]
        xp = x
        x11 = x1
        x22 = x2
        if x2 < x1:
            t = x11
            x11 = x22
            x22 = t
        if xp <= x11 or xp >= x22:
            return -1, [0, 0]
        k2, l2 = line_from_points(x1, y1, x2, y2)
        yp = k2*xp + l2
        if is_in_direction(x, y, xp, yp, angle)==-1:
            return -1, [0, 0]        
        return pp_dist(x, y, xp, yp), [xp, yp]      
            
    k1 = math.tan(math.radians(angle))
    l1 = y - k1*x
    
    if x1 == x2:
        xp = x1
        yp = k1*xp + l1
        if y1 > y2:
            t = y1
            y1 = y2
            y2 = t
        if  y1 <= yp and yp <= y2:
            return pp_dist(x, y, xp, yp), [xp, yp]
        return -1, [0, 0]
    
    
    if y1 == y2:
        yp = y1
        xp = (yp - l1)/k1
        if x1 > x2:
            t = x1
            x1 = x2
            x2 = t
        if  x1 <= xp and xp <= x2:
            return pp_dist(x, y, xp, yp), [xp, yp]
        return -1, [0, 0]
        
    
    k2, l2 = line_from_points(x1, y1, x2, y2)
    
    #x*k1 + l1 = x*k2 + l2
    #print("lines ", k1, l1, k2, l2)
    if (k1-k2) == 0:
        #print("parallel")
        return -1, [0, 0]
    xp = (l2-l1)/(k1-k2)
    if x2 < x1:
        t = x2
        x2 = x1
        x1 = t
    if xp < x1 or x2 < xp:
        return -1, [0, 0]
    yp = k1*xp + l1
    dist = pp_dist(x, y, xp, yp)
    #print("coordinates ",  xp, yp)
    if is_in_direction(x, y, xp, yp, angle)==-1:
        return -1, [0, 0]
    return dist, [xp, yp]

In [16]:
def is_in_direction(x, y, x1, y1, angle):
    if angle >= 45 and angle < 135:
        if y1 < y:
            return -1
    elif angle >= 135 and angle < 225:
        if x1 > x:
            return -1
    elif angle >= 225 and angle < 315:
        if y1 > y:
            return -1  
    else:
        if x1 < x:
            return -1
    return 1

In [17]:
def calc_scan_for_point(x, y, angles, field):
    scan = []
    points = []
    for angle in angles:
        mind = 10000
        point = 0
        for seg in field:
            hp = []
            dist = 0
            #print(angle, seg)
            dist, hp = ps_inter(x, y, angle, seg)
            if is_in_direction(x, y, hp[0], hp[1], angle) == -1:
                dist = -1
            #print( dist, hp)
            #print(" ")
            if dist > -1 and mind > dist:
                mind = dist
                point = hp
                point[0] = int(point[0])
                point[1] = int(point[1])
        scan.append(mind)
        points.append(point)
    return scan, points
                
            

In [56]:
from robocup import simulate_lidar as sim

field = [
    [260, 260, 570, 260],
    [570, 260, 570, 410],
    [570, 410, 660, 520],
    [660, 520, 1160, 520],
    [1160, 520, 1250, 410],
    [1250, 410, 1250, 260],
    [1250, 260, 1560, 260],
    [1560, 260, 1560, 2170],
    [1560, 2170, 1250, 2170],
    [1250, 2170, 1250, 2020],
    [1250, 2020, 1160, 1910],
    [1160, 1910, 660, 1910],
    [660, 1910, 570, 2020],
    [570, 2020, 570, 2170],
    [570, 2170, 260, 2170],
    [260, 2170, 260, 260]
]

ModuleNotFoundError: No module named 'robocup'

In [19]:
w = 500
test_scan = np.zeros((w, w), np.uint8)
x0 = 950
y0 = 500
w = 500
t = 0
for i in range (len(field)):
    x1 = field[i][0]//5 + t
    y1 = field[i][1]//5 + t
    x2 = field[i][2]//5 + t
    y2 = field[i][3]//5 + t
    cv2.line(test_scan,(x1,y1),(x2,y2),255,5)
    
angles = []
for i in range (360):
    angles.append(i)
    
scan, points = calc_scan_for_point(x0, y0, angles, field)
print (scan, points)
for i in range (len(points)):
    point = points[i]
    print(point, angles[i])
    if type(point) is list:
        x1 = point[0]
        y1 = point[1]
        x2 = x0
        y2 = y0
        cv2.line(test_scan,(x1,y1),(x2,y2),255,1)
    else:
        print(angles[i], "ne valja")


cv2.imshow('img', test_scan)
cv2.waitKey(0)
cv2.destroyAllWindows()

[226.36363636363626, 223.2103577495282, 220.20987992668535, 217.35432928163408, 214.63643212656834, 212.04946176220318, 191.3354446701125, 164.11018096250183, 143.70593068655458, 127.84906442999315, 115.17540966287284, 104.816861283357, 96.19468689488264, 88.9082296517159, 82.67130988877491, 77.27406610312562, 72.55910557086598, 68.40607239666534, 64.72135954999584, 61.4310697351449, 58.47608800326175, 55.80856219250669, 53.38934325108032, 51.186093304949104, 49.17186671148478, 47.32403166304998, 45.62344065409717, 44.05378529170539, 42.60108936379028, 41.25330679254633, 39.999999999999936, 38.832080528207065, 37.74159829599715, 36.72156917553331, 35.765832999428014, 34.868935912422, 34.02603233408162, 33.23280282244966, 32.48538490965487, 31.78031458131496, 31.114476537208166, 30.48506173411628, 29.889530997292187, 29.32558371279249, 28.791130792514544, 28.284271247461902, 27.80327182033358, 27.346549221971785, 26.91265459212753, 26.500259866976275, 26.1081457866456, 25.73519131786341

In [20]:
dist, hp = ps_inter(x, y, angles[0], field[0])

NameError: name 'x' is not defined

In [21]:
dist, hp = ps_inter(-10, 10, 180, [0, 0, 0, 500])
dist

-1

In [22]:
is_in_direction(500, 500, 600, 600, 45)

1

In [57]:
from robocup import simulate_lidar as sim
lines = [
    [260, 260, 570, 260],
    [570, 260, 570, 410],
    [570, 410, 660, 520],
    [660, 520, 1160, 520],
    [1160, 520, 1250, 410],
    [1250, 410, 1250, 260],
    [1250, 260, 1560, 260],
    [1560, 260, 1560, 2170],
    [1560, 2170, 1250, 2170],
    [1250, 2170, 1250, 2020],
    [1250, 2020, 1160, 1910],
    [1160, 1910, 660, 1910],
    [660, 1910, 570, 2020],
    [570, 2020, 570, 2170],
    [570, 2170, 260, 2170],
    [260, 2170, 260, 260]
]

ModuleNotFoundError: No module named 'robocup'

In [55]:
def max_speeds(x1, y1):
    angles2 = [180, 270, 0, 90]
    scan, points = calc_scan_for_point(x1, y1, angles2, lines)
    return scan

In [54]:
max_speeds(415, 270)

[155, 10, 155, 1900]

In [31]:
w = 500
x0 = y0 = 0
t = 0 
for i in range (len(field)):
    for j in range (len(field[i])):
        field[i][j] //= 5
while (x0 < w and y0 < w):
    test_scan = np.zeros((w, w), np.uint8)
    for i in range (len(field)):
        x1 = field[i][0] + t
        y1 = field[i][1] + t
        x2 = field[i][2] + t
        y2 = field[i][3] + t
        cv2.line(test_scan,(x1,y1),(x2,y2),255,5)
    scan, points = calc_scan_for_point(x0, y0, angles, field)
    for i in range (len(points)):
        point = points[i]
        if type(point) is list:
            x1 = point[0]
            y1 = point[1]
            x2 = x0
            y2 = y0
            cv2.line(test_scan,(x1,y1),(x2,y2),255,1)
        else:
            ad = 0
            #print(angles[i], "ne valja",point)

    x0 += 1
    y0 += 1
    cv2.imshow('img', test_scan)
    if cv2.waitKey(1) != -1:
        print(cv2.waitKey(1))
        break
    time.sleep(0.01)
cv2.destroyAllWindows()

-1


In [9]:
lidar.get_info()


{'model': 40,
 'firmware': (1, 25),
 'hardware': 5,
 'serialnumber': '85E39AF2C1EA9FC2A2EB92F116303C00'}